In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import trading_vix_and_spy_utils

In [2]:
data_names = []
data_names.append("./hourly_data/VIX_1300D_1hour_TRADES.csv")
data_names.append("./hourly_data/VIX_1300D_1hour_OPTION_IMPLIED_VOLATILITY.csv")
data_names.append("./hourly_data/VIXY_1300D_1hour_BID.csv")
data_names.append("./hourly_data/VIXY_1300D_1hour_ASK.csv")
data_names.append("./hourly_data/VIXY_1300D_1hour_OPTION_IMPLIED_VOLATILITY.csv")
data_names.append("./hourly_data/SPY_1300D_1hour_BID.csv")
data_names.append("./hourly_data/SPY_1300D_1hour_ASK.csv")
data_names.append("./hourly_data/SPY_1300D_1hour_OPTION_IMPLIED_VOLATILITY.csv")
new_column_names = []
new_column_names.append('vix_index')
new_column_names.append('vix_index_iv')
new_column_names.append('vixy_bid')
new_column_names.append('vixy_ask')
new_column_names.append('vixy_iv')
new_column_names.append('spy_bid')
new_column_names.append('spy_ask')
new_column_names.append('spy_iv')

In [3]:
#combine data

for index in range(0,len(data_names)):
    
    data_name = data_names[index]
    new_column_name = new_column_names[index]
    
    if index == 0:
        
        total_data = pd.read_csv(data_name)
        total_data = total_data.rename(columns = {"date":"date",\
                                "open":new_column_name+"_open",\
                                "high":new_column_name+'_high',\
                                'low':new_column_name+'_low',\
                                'close':new_column_name+'_close',\
                                'volume':new_column_name+'_volume'})
    
    else:
        
        new_data = pd.read_csv(data_name)
        new_data = new_data.rename(columns = {"date":"date",\
                                "open":new_column_name+"_open",\
                                "high":new_column_name+'_high',\
                                'low':new_column_name+'_low',\
                                'close':new_column_name+'_close',\
                                'volume':new_column_name+'_volume'})
        
        total_data = pd.merge(total_data,new_data,on = 'date',how = 'inner')

In [4]:
distiled_column_names = []
for entry in total_data.columns:
    if entry == "date":
        distiled_column_names.append(entry)
    elif 'close' in entry:
        distiled_column_names.append(entry)
        
total_data_distiled = pd.DataFrame()
for entry in distiled_column_names:
    total_data_distiled[entry] = total_data[entry]

In [5]:
total_data_distiled.head()

,date,vix_index_close,vix_index_iv_close,vixy_bid_close,vixy_ask_close,vixy_iv_close,spy_bid_close,spy_ask_close,spy_iv_close
0,20171212 09:30:00,9.61,0.811140,96.64,96.68,0.754611,263.90,263.91,0.073118
1,20171212 10:00:00,9.42,0.816331,96.76,96.80,0.714940,264.08,264.09,0.072404
2,20171212 11:00:00,9.43,0.835729,96.84,96.88,0.531240,264.42,264.43,0.071483
3,20171212 12:00:00,9.36,0.836285,96.84,96.88,0.529145,264.43,264.44,0.069959
4,20171212 13:00:00,9.69,0.840460,97.24,97.32,0.556671,264.23,264.25,0.071562


In [6]:
total_data_distiled.shape

(6191, 9)

In [7]:
total_data_distiled.columns.tolist()

['date',
 'vix_index_close',
 'vix_index_iv_close',
 'vixy_bid_close',
 'vixy_ask_close',
 'vixy_iv_close',
 'spy_bid_close',
 'spy_ask_close',
 'spy_iv_close']

In [8]:
index_begin_at = total_data_distiled.columns.tolist().index('vix_index_close')+1


window_lengths = [10,30,50,100,200]

for index in range(0,len(window_lengths)):
    window_length = window_lengths[index]
    new_col_index = index_begin_at+index #vix_index_close originally has column index 1
    new_col_name = 'vix_index_'+str(window_length)+"_slope"
    
    new_data = trading_vix_and_spy_utils.compute_slope_feature(total_data_distiled,'vix_index_close',window_length)
    total_data_distiled.insert(loc = new_col_index,column = new_col_name,value = new_data)


In [9]:
total_data_distiled.columns.tolist()

['date',
 'vix_index_close',
 'vix_index_10_slope',
 'vix_index_30_slope',
 'vix_index_50_slope',
 'vix_index_100_slope',
 'vix_index_200_slope',
 'vix_index_iv_close',
 'vixy_bid_close',
 'vixy_ask_close',
 'vixy_iv_close',
 'spy_bid_close',
 'spy_ask_close',
 'spy_iv_close']

In [10]:
#build features for vix based on vix6 trading environment
#compute the exponential moving average
mv_10 = total_data_distiled['vix_index_close'].ewm(span = 10).mean()
mv_20 = total_data_distiled['vix_index_close'].ewm(span = 20).mean()
mv_30 = total_data_distiled['vix_index_close'].ewm(span = 30).mean()
mv_50 = total_data_distiled['vix_index_close'].ewm(span = 50).mean()
mv_100 = total_data_distiled['vix_index_close'].ewm(span = 100).mean()

spot_to_mv_10 = total_data_distiled['vix_index_close']/mv_10
spot_to_mv_20 = total_data_distiled['vix_index_close']/mv_20
spot_to_mv_30 = total_data_distiled['vix_index_close']/mv_30
spot_to_mv_50 = total_data_distiled['vix_index_close']/mv_50
spot_to_mv_100 = total_data_distiled['vix_index_close']/mv_100

vix_measure = spot_to_mv_10+spot_to_mv_20+spot_to_mv_30+spot_to_mv_50+spot_to_mv_100
vix_measure_list = vix_measure.tolist()

index_begin_at = total_data_distiled.columns.tolist().index('vix_index_iv_close')+1
total_data_distiled.insert(loc = index_begin_at,column = 'vix_index_mv_ratio',value = vix_measure_list)

index_begin_at = total_data_distiled.columns.tolist().index('vix_index_mv_ratio')+1
threshold_list = [5.5,6,6.5]
col_index_adder = 0
for threshold in threshold_list:
    counting_days = trading_vix_and_spy_utils.day_counter_helper(vix_measure_list,threshold)
    total_data_distiled.insert(loc = index_begin_at+col_index_adder,column = 'vix_index_intervals_since_'+str(threshold),value = counting_days)
    col_index_adder += 1

In [11]:
total_data_distiled.columns.tolist()

['date',
 'vix_index_close',
 'vix_index_10_slope',
 'vix_index_30_slope',
 'vix_index_50_slope',
 'vix_index_100_slope',
 'vix_index_200_slope',
 'vix_index_iv_close',
 'vix_index_mv_ratio',
 'vix_index_intervals_since_5.5',
 'vix_index_intervals_since_6',
 'vix_index_intervals_since_6.5',
 'vixy_bid_close',
 'vixy_ask_close',
 'vixy_iv_close',
 'spy_bid_close',
 'spy_ask_close',
 'spy_iv_close']

In [12]:
vixy_mid_close = (total_data_distiled['vixy_bid_close']+total_data_distiled['vixy_ask_close'])/2
index_begin_at = total_data_distiled.columns.tolist().index('vixy_ask_close')+1
total_data_distiled.insert(loc = index_begin_at,column = 'vixy_mid_close',value = vixy_mid_close)

index_begin_at = total_data_distiled.columns.tolist().index('vix_index_intervals_since_6.5')+1
window_lengths = [10,30,50,100,200]

for index in range(0,len(window_lengths)):
    window_length = window_lengths[index]
    new_col_index = index_begin_at+index #vix_index_close originally has column index 1
    new_col_name = 'vixy_mid_close_'+str(window_length)+"_slope"
    
    new_data = trading_vix_and_spy_utils.compute_slope_feature(total_data_distiled,'vixy_mid_close',window_length)
    total_data_distiled.insert(loc = new_col_index,column = new_col_name,value = new_data)


In [13]:
total_data_distiled.columns.tolist()

['date',
 'vix_index_close',
 'vix_index_10_slope',
 'vix_index_30_slope',
 'vix_index_50_slope',
 'vix_index_100_slope',
 'vix_index_200_slope',
 'vix_index_iv_close',
 'vix_index_mv_ratio',
 'vix_index_intervals_since_5.5',
 'vix_index_intervals_since_6',
 'vix_index_intervals_since_6.5',
 'vixy_mid_close_10_slope',
 'vixy_mid_close_30_slope',
 'vixy_mid_close_50_slope',
 'vixy_mid_close_100_slope',
 'vixy_mid_close_200_slope',
 'vixy_bid_close',
 'vixy_ask_close',
 'vixy_mid_close',
 'vixy_iv_close',
 'spy_bid_close',
 'spy_ask_close',
 'spy_iv_close']

In [14]:
spy_mid_close = (total_data_distiled['spy_bid_close']+total_data_distiled['spy_ask_close'])/2
index_begin_at = total_data_distiled.columns.tolist().index('spy_ask_close')+1
total_data_distiled.insert(loc = index_begin_at,column = 'spy_mid_close',value = spy_mid_close)

index_begin_at = total_data_distiled.columns.tolist().index('vixy_mid_close_200_slope')+1
window_lengths = [10,30,50,100,200]

for index in range(0,len(window_lengths)):
    window_length = window_lengths[index]
    new_col_index = index_begin_at+index #vix_index_close originally has column index 1
    new_col_name = 'spy_mid_close_'+str(window_length)+"_slope"
    
    new_data = trading_vix_and_spy_utils.compute_slope_feature(total_data_distiled,'spy_mid_close',window_length)
    total_data_distiled.insert(loc = new_col_index,column = new_col_name,value = new_data)


In [15]:
total_data_distiled.columns.tolist()

['date',
 'vix_index_close',
 'vix_index_10_slope',
 'vix_index_30_slope',
 'vix_index_50_slope',
 'vix_index_100_slope',
 'vix_index_200_slope',
 'vix_index_iv_close',
 'vix_index_mv_ratio',
 'vix_index_intervals_since_5.5',
 'vix_index_intervals_since_6',
 'vix_index_intervals_since_6.5',
 'vixy_mid_close_10_slope',
 'vixy_mid_close_30_slope',
 'vixy_mid_close_50_slope',
 'vixy_mid_close_100_slope',
 'vixy_mid_close_200_slope',
 'spy_mid_close_10_slope',
 'spy_mid_close_30_slope',
 'spy_mid_close_50_slope',
 'spy_mid_close_100_slope',
 'spy_mid_close_200_slope',
 'vixy_bid_close',
 'vixy_ask_close',
 'vixy_mid_close',
 'vixy_iv_close',
 'spy_bid_close',
 'spy_ask_close',
 'spy_mid_close',
 'spy_iv_close']

In [16]:
total_data_distiled = total_data_distiled[['date',
 'vix_index_close',
 'vix_index_10_slope',
 'vix_index_30_slope',
 'vix_index_50_slope',
 'vix_index_100_slope',
 'vix_index_200_slope',
 'vix_index_iv_close',
 'vix_index_mv_ratio',
 'vix_index_intervals_since_5.5',
 'vix_index_intervals_since_6',
 'vix_index_intervals_since_6.5',
 'vixy_mid_close_10_slope',
 'vixy_mid_close_30_slope',
 'vixy_mid_close_50_slope',
 'vixy_mid_close_100_slope',
 'vixy_mid_close_200_slope',
 'vixy_iv_close',
 'spy_mid_close_10_slope',
 'spy_mid_close_30_slope',
 'spy_mid_close_50_slope',
 'spy_mid_close_100_slope',
 'spy_mid_close_200_slope',
 'spy_iv_close',
 'vixy_bid_close',
 'vixy_ask_close',
 'vixy_mid_close',
 'spy_bid_close',
 'spy_ask_close',
 'spy_mid_close',
        ]]

In [17]:
total_data_distiled

,date,vix_index_close,vix_index_10_slope,vix_index_30_slope,vix_index_50_slope,vix_index_100_slope,vix_index_200_slope,vix_index_iv_close,vix_index_mv_ratio,vix_index_intervals_since_5.5,...,spy_mid_close_50_slope,spy_mid_close_100_slope,spy_mid_close_200_slope,spy_iv_close,vixy_bid_close,vixy_ask_close,vixy_mid_close,spy_bid_close,spy_ask_close,spy_mid_close
0,20171212 09:30:00,9.61,0.000000,0.000000,0.000000,0.000000,0.000000,0.811140,5.000000,0,...,0.000000,0.000000,0.000000,0.073118,96.64,96.68,96.660,263.90,263.91,263.905
1,20171212 10:00:00,9.42,0.000000,0.000000,0.000000,0.000000,0.000000,0.816331,4.952189,1,...,0.000000,0.000000,0.000000,0.072404,96.76,96.80,96.780,264.08,264.09,264.085
2,20171212 11:00:00,9.43,0.000000,0.000000,0.000000,0.000000,0.000000,0.835729,4.972748,2,...,0.000000,0.000000,0.000000,0.071483,96.84,96.88,96.860,264.42,264.43,264.425
3,20171212 12:00:00,9.36,0.000000,0.000000,0.000000,0.000000,0.000000,0.836285,4.953797,3,...,0.000000,0.000000,0.000000,0.069959,96.84,96.88,96.860,264.43,264.44,264.435
4,20171212 13:00:00,9.69,0.000000,0.000000,0.000000,0.000000,0.000000,0.840460,5.097437,4,...,0.000000,0.000000,0.000000,0.071562,97.24,97.32,97.280,264.23,264.25,264.240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6186,20210702 11:00:00,14.55,-0.139636,-0.042031,-0.013891,-0.027053,-0.012845,0.999332,4.643858,68,...,0.148079,0.122679,0.066142,0.102899,23.06,23.07,23.065,432.44,432.45,432.445
6187,20210702 12:00:00,14.65,-0.135576,-0.048211,-0.014809,-0.028419,-0.012657,0.990665,4.694015,69,...,0.152996,0.126725,0.067156,0.103010,23.16,23.17,23.165,433.26,433.27,433.265
6188,20210702 13:00:00,14.82,-0.121030,-0.051657,-0.015032,-0.029504,-0.012550,0.996983,4.760929,70,...,0.156730,0.130460,0.068182,0.103200,23.40,23.41,23.405,433.31,433.32,433.315
6189,20210702 14:00:00,14.92,-0.091394,-0.052618,-0.015521,-0.030314,-0.012372,1.001602,4.802011,71,...,0.160375,0.133861,0.069007,0.104200,23.35,23.36,23.355,433.26,433.27,433.265


In [18]:
#build spy observation
spy_data_per_day = 7 #this is a parameter that we can tune
spy_max_observation_history = 40 #this is a parameter that we can tune
spy_temp_data_max_rows = total_data_distiled.shape[0]-spy_max_observation_history*spy_data_per_day+1
spy_temp_data = np.zeros((spy_temp_data_max_rows,spy_max_observation_history*spy_data_per_day))

for end_interval_index in range(total_data_distiled.shape[0]-spy_temp_data_max_rows+1,total_data.shape[0]+1):
    price_history = total_data_distiled['spy_mid_close'][end_interval_index-spy_data_per_day*spy_max_observation_history\
                                                    :end_interval_index]
    spy_temp_data[end_interval_index-spy_data_per_day*spy_max_observation_history,:] = price_history
    
spy_observation_data_list = []
#fill zeros for the spy observation data list
for _ in range(0,spy_max_observation_history*spy_data_per_day-1):
    spy_observation_data_list.append([0])
    
for row_index in range(0,spy_temp_data.shape[0]):
    spy_observation_data_list.append(spy_temp_data[row_index,:].tolist())


In [19]:
total_data_distiled['spy_mid_close'] = spy_observation_data_list

In [20]:
total_data_distiled = total_data_distiled.drop('vixy_mid_close', 1)
#where 1 is the axis number (0 for rows and 1 for columns.)

In [21]:
total_data_distiled = total_data_distiled.iloc[300:] #to get rid of zeros

In [22]:
total_data_distiled.shape

(5891, 29)

In [23]:
total_data_distiled = total_data_distiled.reset_index(drop=True)

In [24]:
total_data_distiled.to_csv('full_feature_dataframe.csv')

In [25]:
#the second entry of result is the p value of the adf test
#haveing a small value means the time series is stationary
#this means we get mean reversion

from statsmodels.tsa.stattools import adfuller

for column_name in total_data_distiled.columns.tolist():
    
    if column_name != 'date' and column_name != 'spy_mid_close':
        print('the column name is',column_name)
        result = adfuller(total_data_distiled[column_name])
        print(result)
        print(' ')

the column name is vix_index_close
(-3.0328022617104597, 0.031942570809399586, 34, 5856, {'1%': -3.4314671736301348, '5%': -2.862033685824235, '10%': -2.5670327868304237}, 15510.27449461536)
 
the column name is vix_index_10_slope
(-11.790661440693887, 9.844454996140919e-22, 33, 5857, {'1%': -3.4314669828048268, '5%': -2.862033601513211, '10%': -2.567032741949302}, -17359.492845033747)
 
the column name is vix_index_30_slope
(-9.045817899421898, 4.99034349976091e-15, 34, 5856, {'1%': -3.4314671736301348, '5%': -2.862033685824235, '10%': -2.5670327868304237}, -41209.664543990846)
 
the column name is vix_index_50_slope
(-7.5523324362082205, 3.1646880712286825e-11, 33, 5857, {'1%': -3.4314669828048268, '5%': -2.862033601513211, '10%': -2.567032741949302}, -51555.73332273364)
 
the column name is vix_index_100_slope
(-7.43630446774802, 6.170458300504741e-11, 33, 5857, {'1%': -3.4314669828048268, '5%': -2.862033601513211, '10%': -2.567032741949302}, -67142.00943652475)
 
the column name is

In [26]:
total_data_distiled.head()

,date,vix_index_close,vix_index_10_slope,vix_index_30_slope,vix_index_50_slope,vix_index_100_slope,vix_index_200_slope,vix_index_iv_close,vix_index_mv_ratio,vix_index_intervals_since_5.5,...,spy_mid_close_30_slope,spy_mid_close_50_slope,spy_mid_close_100_slope,spy_mid_close_200_slope,spy_iv_close,vixy_bid_close,vixy_ask_close,spy_bid_close,spy_ask_close,spy_mid_close
0,20180213 15:00:00,25.25,-0.095939,-0.174810,0.070209,0.236974,0.107940,1.390178,4.930738,32,...,0.063217,-0.178645,-0.291841,-0.041100,0.173715,158.96,159.08,266.01,266.02,"[265.505, 265.925, 266.505, 266.615, 266.795, ..."
1,20180214 09:30:00,21.00,-0.352061,-0.249081,0.029248,0.231797,0.107818,1.450803,4.166463,33,...,0.114150,-0.156184,-0.291692,-0.043478,0.163285,154.04,154.24,265.07,265.09,"[265.925, 266.505, 266.615, 266.795, 266.815, ..."
2,20180214 10:00:00,19.98,-0.601515,-0.324445,-0.012006,0.225953,0.107514,1.284533,4.034567,34,...,0.169808,-0.131902,-0.290362,-0.045604,0.154491,149.68,149.76,266.99,267.01,"[266.505, 266.615, 266.795, 266.815, 266.525, ..."
3,20180214 11:00:00,20.21,-0.769939,-0.398294,-0.056074,0.220379,0.107207,1.232751,4.143761,35,...,0.220507,-0.103696,-0.289011,-0.047699,0.156285,149.28,149.32,266.93,266.94,"[266.615, 266.795, 266.815, 266.525, 268.445, ..."
4,20180214 12:00:00,19.34,-0.845515,-0.454065,-0.101014,0.214151,0.106767,1.270389,4.032420,36,...,0.264898,-0.074093,-0.286551,-0.049585,0.149649,148.20,148.24,268.10,268.12,"[266.795, 266.815, 266.525, 268.445, 268.265, ..."
